In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import mplleaflet
from geopy.distance import geodesic

In [5]:
#import data
wyp2018 = pd.read_csv('../../dane/wyp2018_enriched.csv')
stacje2018 = pd.read_csv('../../dane/stacje_2018.csv')

In [6]:
#merge datasets adding geo coordinates to departure station
df_temp = pd.merge(wyp2018, stacje2018, left_on='departure', right_on='name')

In [7]:
#merge datasets adding geo coordinates to return station
df = pd.merge(df_temp,stacje2018, left_on='return', right_on='name')

In [8]:
#remove unnecessary columns, rename ambigous
df = df.drop(columns = ['Unnamed: 0', 'name_x', 'name_y', 'capacity_x', 'capacity_y'])
df = df.rename(columns={'id_x': 'dep_id', 'lat_x': 'dep_lat', 'lon_x' : 'dep_lon', 'id_y' : 'ret_id', 'lat_y' : 'ret_lat', 'lon_y' : 'ret_lon'})

In [9]:
#sort df by 'start_time' column
df = df.sort_values(by=['start_time'])

In [10]:
#check how many stations were in Katowice
stacje2018['name'].count()

54

In [11]:
#plot them on map

#Create a new figure.
fig = plt.figure(figsize=(20,10))

#A scatter plot of *y* vs *x* with varying marker size and/or color.
plt.scatter(stacje2018['lon'], stacje2018['lat'], color='red', s=50)

#Convert a Matplotlib Figure to a Leaflet map. Embed in IPython notebook.
mplleaflet.display(fig=fig)

C:\Users\acibis\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\display.py:694: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [21]:
#plot stations' capacity on a map

#Create a new figure.
fig = plt.figure(figsize=(20,10))

#A scatter plot of *y* vs *x* with varying marker size and/or color.
plt.scatter(stacje2018['lon'], stacje2018['lat'], c=stacje2018['capacity'], s=stacje2018['capacity']*50)

#Convert a Matplotlib Figure to a Leaflet map. Embed in IPython notebook.
mplleaflet.display(fig=fig)


In [22]:
#how many bikes depart from each station monthly?
departures = df.groupby(['departure','start_month'])['bike_num'].count()
deps_month=pd.DataFrame(departures)
deps_month = deps_month.unstack(level=1)
deps_month.head(5)

bike_num                                         \
start_month                         4       5       6       7       8      9    
departure                                                                       
Al. Bolesława Krzywoustego      1293.0  1337.0  1093.0  1293.0  1363.0  688.0   
Al. Księcia Henryka Pobożnego   1275.0  1300.0  1064.0   992.0  1157.0  638.0   
Al. Księżnej Jadwigi Śląskiej    709.0   718.0   689.0   558.0   728.0  524.0   
Armii Krajowej/Jankego           258.0   345.0   318.0   286.0   308.0  198.0   
Bogucice Szpital                 382.0   481.0   367.0   378.0   445.0  200.0   

                                                   
start_month                       10     11    12  
departure                                          
Al. Bolesława Krzywoustego     321.0  118.0  20.0  
Al. Księcia Henryka Pobożnego  297.0   95.0  25.0  
Al. Księżnej Jadwigi Śląskiej  152.0   83.0   5.0  
Armii Krajowej/Jankego         141.0   42.0   2.0  
Bogucice Szpital               129.0   59.0   1.0

In [23]:
#how many bikes return to each station monthly?
returns = df.groupby(['return','start_month'])['bike_num'].count()
reps_month=pd.DataFrame(returns)
reps_month = reps_month.unstack(level=1)
reps_month.head(5)

bike_num                                         \
start_month                         4       5       6       7       8      9    
return                                                                          
Al. Bolesława Krzywoustego      1337.0  1402.0  1235.0  1447.0  1499.0  784.0   
Al. Księcia Henryka Pobożnego   1294.0  1330.0  1116.0  1028.0  1220.0  679.0   
Al. Księżnej Jadwigi Śląskiej    703.0   698.0   742.0   566.0   740.0  542.0   
Armii Krajowej/Jankego           270.0   325.0   351.0   302.0   310.0  189.0   
Bogucice Szpital                 385.0   471.0   356.0   371.0   471.0  227.0   

                                                   
start_month                       10     11    12  
return                                             
Al. Bolesława Krzywoustego     367.0  150.0  12.0  
Al. Księcia Henryka Pobożnego  309.0  117.0  25.0  
Al. Księżnej Jadwigi Śląskiej  161.0   82.0   6.0  
Armii Krajowej/Jankego         140.0   46.0   5.0  
Bogucice Szpital               150.0   48.0   9.0

In [25]:
#find the difference
(deps_month - reps_month).head(5)

bike_num                                         \
start_month                         4     5      6      7      8     9     10   
departure                                                                       
Al. Bolesława Krzywoustego       -44.0 -65.0 -142.0 -154.0 -136.0 -96.0 -46.0   
Al. Księcia Henryka Pobożnego    -19.0 -30.0  -52.0  -36.0  -63.0 -41.0 -12.0   
Al. Księżnej Jadwigi Śląskiej      6.0  20.0  -53.0   -8.0  -12.0 -18.0  -9.0   
Armii Krajowej/Jankego           -12.0  20.0  -33.0  -16.0   -2.0   9.0   1.0   
Bogucice Szpital                  -3.0  10.0   11.0    7.0  -26.0 -27.0 -21.0   

                                          
start_month                      11   12  
departure                                 
Al. Bolesława Krzywoustego    -32.0  8.0  
Al. Księcia Henryka Pobożnego -22.0  0.0  
Al. Księżnej Jadwigi Śląskiej   1.0 -1.0  
Armii Krajowej/Jankego         -4.0 -3.0  
Bogucice Szpital               11.0 -8.0

In [26]:
#find most rented bike
mr_bike_id = df.groupby('bike_num')['start_time'].count().sort_values(ascending=False).head(1).index[0]

In [43]:
#does the busiest bike visit all the stations during te season?
#find all its rides
mr_bike = df[df['bike_num'] == mr_bike_id]

#find the number of visited stations
d = mr_bike.groupby('departure').count().index.unique()
r = mr_bike.groupby('return').count().index.unique()
r.union(d).nunique()

#count the difference


42

In [ ]:
#what were the not visited stations?
#find all its rides
mr_bike = df[df['bike_num'] == mr_bike_id]
#find all the stations the bike has visited:
a_stations = mr_bike.groupby('departure').count().index.to_list()
b_stations = mr_bike.groupby('return').count().index.to_list()
stats = list(set(a_stations) | set(b_stations))

#find all the stations available
alls = stacje2018['name'].to_list()

#find the difference
not_visited = (list(set(alls) - set(stats)))
not_visited